<a href="https://colab.research.google.com/github/Siri2191/portifolio/blob/main/Clustering_with_Spark_Spotify_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objetivo: Construir um sistema de recomendação de musica usando o spark
---

## Ferramentas:
---


* https://spark.apache.org/docs/latest/api/python/index.html 
* https://spark.apache.org/docs/latest/ml-guide.html

## Fonte e Dicionário dos dados:
---

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features

* `Acousticness/Acústica:` Variável numérica, medida de confiança de 0,0 a 1,0 se a faixa é acústica. 1.0 representa alta confiança de que a faixa é acústica.

* `Danceability/Dançabilidade:` Variável numérica, a dançabilidade descreve o quão adequada uma faixa é para dançar com base em uma combinação de elementos musicais, incluindo tempo, estabilidade do ritmo, força da batida e regularidade geral. Um valor de 0,0 é o menos dançável e 1,0 é o mais dançável.

* `Duration_ms:`Variável numérica, a duração da trilha em milissegundos.

* `Duration_min:` Variável numérica, a duração da faixa em minutos.

* `Energy/Energia:` Variável numérica, Energia é uma medida de 0,0 a 1,0 e representa uma medida perceptiva de intensidade e atividade. Normalmente, as faixas energéticas parecem rápidas, altas e barulhentas. Por exemplo, o death metal tem alta energia, enquanto um prelúdio de Bach tem uma pontuação baixa na escala. As características perceptivas que contribuem para este atributo incluem faixa dinâmica, intensidade percebida, timbre, taxa de início e entropia geral.

* `Explicit/Explícito:` Variável categórica, se a faixa tem ou não letras explícitas (verdadeiro = sim (1); falso = não(0), não OU desconhecido).

* `Id:` O ID do Spotify para a faixa.

* `Instrumentalness/Instrumentalidade:` Variável numérica, prevê se uma faixa não contém vocais. Os sons “Ooh” e “aah” são tratados como instrumentais neste contexto. Faixas de rap ou de palavras faladas são claramente “vocais”. Quanto mais próximo o valor de instrumentalidade estiver de 1,0, maior a probabilidade de a faixa não conter conteúdo vocal. Valores acima de 0,5 destinam-se a representar faixas instrumentais, mas a confiança é maior à medida que o valor se aproxima de 1,0.

* `Key/Chave:`Variável numérica, a chave geral estimada da faixa. Os inteiros são mapeados para pitchs usando a notação padrão de Pitch Class. Por exemplo. 0 = C, 1 = C#/Db, 2 = D, e assim por diante. Se nenhuma chave foi detectada, o valor é -1.

* `Liveness/ Ao vivo:` Variável numérica, detecta a presença de um público na gravação. Valores mais altos de vivacidade representam uma probabilidade maior de que a faixa tenha sido executada ao vivo. Um valor acima de 0,8 fornece uma forte probabilidade de que a faixa esteja ativa.

* `Loudness/ Volume em dB:` Variável numérica, volume geral de uma faixa em decibéis (dB). Os valores de volume são calculados em média em toda a faixa e são úteis para comparar o volume relativo das faixas. A sonoridade é a qualidade de um som que é o principal correlato psicológico da força física (amplitude). Os valores típicos variam entre -60 e 0 db.

* `Mode/ Modo:` Variável numérica, o modo indica a modalidade (maior ou menor) de uma faixa, o tipo de escala da qual seu conteúdo melódico é derivado. Maior é representado por 1 e menor é 0.

* `Popularity/Popularidade:` Variável numérica, a popularidade de uma faixa é um valor entre 0 e 100, sendo 100 o mais popular. A popularidade é calculada por algoritmo e é baseada, em grande parte, no número total de execuções que a faixa teve e quão recentes são essas execuções.

* `Speechiness/Fala:` Variável numérica, a fala detecta a presença de palavras faladas em uma faixa. Quanto mais exclusivamente falada a gravação (por exemplo, talk show, audiolivro, poesia), mais próximo de 1,0 o valor do atributo. Valores acima de 0,66 descrevem faixas que provavelmente são feitas inteiramente de palavras faladas. Valores entre 0,33 e 0,66 descrevem faixas que podem conter música e fala, seja em seções ou em camadas, incluindo casos como música rap. Os valores abaixo de 0,33 provavelmente representam músicas e outras faixas que não são de fala.

* `Tempo:` Variável numérica, Tempo estimado geral de uma faixa em batidas por minuto (BPM). Na terminologia musical, tempo é a velocidade ou ritmo de uma determinada peça e deriva diretamente da duração média da batida.

* `Valence/Valência:` Variável numérica, Medida de 0,0 a 1,0 descrevendo a positividade musical transmitida por uma faixa. Faixas com alta valência soam mais positivas (por exemplo, feliz, alegre, eufórica), enquanto faixas com baixa valência soam mais negativas (por exemplo, triste, deprimida, irritada).

* `Year/Ano:` Ano em que a música foi lançada.

<h1>Instalando o PySpark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 199 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3c53d6d941e2d9cd0e941bda6f9cda15527575908a5c6a5750d525c01d0240c7
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Importando a Classe SparkSession que é por onde vamos interagir com o Spark

In [2]:
from pyspark.sql import SparkSession

Criando a sessão Spark, aqui podemos verificar a versão que estamos utilizando e o nome da nossa App Spark

In [4]:
sessao_spark = SparkSession.builder\
                           .appName("Recomendador PySpark")\
                           .getOrCreate()
sessao_spark

## Leitura dos dados:
---

In [5]:
from pyspark import SparkFiles

In [6]:
url_dados ='https://github.com/IgorNascAlves/dados/blob/main/dados_musicas.csv?raw=true'

sessao_spark.sparkContext.addFile(url_dados) # adiciona o arquivo na sessoa spark

path_dados_file = 'file://' + SparkFiles.get('dados_musicas.csv')  # cria o caminho para leitura

In [7]:
dados = sessao_spark.read.csv(path_dados_file, header = True, sep=';',inferSchema= True)
dados.show()

+------------------+----+------------+--------------+------------------+-----------+------------------+--------+--------------------+--------------------+---+--------+-------------------+----+--------------------+----------+-----------+------------------+--------------------+
|           valence|year|acousticness|       artists|      danceability|duration_ms|            energy|explicit|                  id|    instrumentalness|key|liveness|           loudness|mode|                name|popularity|speechiness|             tempo|        artists_song|
+------------------+----+------------+--------------+------------------+-----------+------------------+--------+--------------------+--------------------+---+--------+-------------------+----+--------------------+----------+-----------+------------------+--------------------+
|             0.285|2000|     0.00239|      Coldplay|             0.429|     266773|0.6609999999999999|       0|3AJwUDP919kvQ9Qco...|             1.21E-4| 11|   0.234|  

## Uma primeira olhada nos dados:
---

In [8]:
dados.printSchema()

root
 |-- valence: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: double (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- artists_song: string (nullable = true)



In [9]:
dados.count()

20311

In [10]:
len(dados.columns)

19

In [11]:
import pyspark.sql.functions as f

In [12]:
dados.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in dados.columns]).show()

+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+
|valence|year|acousticness|artists|danceability|duration_ms|energy|explicit| id|instrumentalness|key|liveness|loudness|mode|name|popularity|speechiness|tempo|artists_song|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+
|      0|   0|           0|      0|           0|          0|     0|       0|  0|               0|  0|       0|       0|   0|   0|         0|          0|    0|           0|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+



In [13]:
print(sorted(dados.select('year').distinct().collect()))

[Row(year=2000), Row(year=2001), Row(year=2002), Row(year=2003), Row(year=2004), Row(year=2005), Row(year=2006), Row(year=2007), Row(year=2008), Row(year=2009), Row(year=2010), Row(year=2011), Row(year=2012), Row(year=2013), Row(year=2014), Row(year=2015), Row(year=2016), Row(year=2017), Row(year=2018), Row(year=2019), Row(year=2020)]


## Comportamento das musicas ao longo dos anos:
---

In [14]:
url_anos_dados = 'https://github.com/IgorNascAlves/dados/blob/main/dados_musicas_ano.csv?raw=true'

sessao_spark.sparkContext.addFile(url_anos_dados)

path_dados_file = 'file://'+ SparkFiles.get('dados_musicas_ano.csv')

dados_anos = sessao_spark.read.csv(path_dados_file, header =True, inferSchema=True)

dados_anos.show()

+----+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+---+
|mode|year|      acousticness|       danceability|       duration_ms|             energy|   instrumentalness|           liveness|           loudness|        speechiness|             tempo|            valence|         popularity|key|
+----+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+---+
|   1|1921|0.8868960000000005| 0.4185973333333336|260537.16666666663|0.23181513333333334|0.34487805886666656|            0.20571| -17.04866666666665|           0.073662|101.53149333333329|0.37932666666666665| 0.6533333333333333|  2|
|   1|1922|0.9385915492957748| 0.4820422535211267|165469.74647887325

In [15]:
dados_anos = dados_anos.filter('year >= 2000')
dados_anos.show()

+----+----+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+---+
|mode|year|       acousticness|      danceability|       duration_ms|            energy|   instrumentalness|           liveness|           loudness|        speechiness|             tempo|           valence|        popularity|key|
+----+----+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+---+
|   1|2000|0.28932270051635994| 0.590918047034764| 242724.6426380368|0.6254128323108387|0.10116776879345596| 0.1976860429447853| -8.247765848670758|0.08920541922290394| 118.9993231083843|0.5594754601226991|  46.6840490797546|  7|
|   1|2001| 0.2868424748428934|0.5833178553615969|240307.79600997505|0.626985522

In [16]:
dados_anos.count()

21

In [17]:
len(dados_anos.columns)

14

In [18]:
import plotly.express as px

#O plotly nao suporta dataframes no formato spark, por isso transformamos para formato pandas com to_pandas. mas isso só funciona pq a base é pequena

fig = px.line(dados_anos.toPandas(), x='year', y='loudness', markers = True, title = 'Variação do loudness ao longo dos anos')  
fig.show()

In [19]:
import plotly.graph_objects as go

fig = go.Figure()

temp = dados_anos.toPandas()

fig.add_trace(go.Scatter(x = temp['year'], y=temp['acousticness'], name = 'acousticness'))

fig.show()

In [20]:

fig.add_trace(go.Scatter(x=temp['year'], y=temp['acousticness'], name='Acousticness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['valence'],
                    name='Valence'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['danceability'],
                    name='Danceability'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['energy'],
                    name='Energy'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['instrumentalness'],
                    name='Instrumentalness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['liveness'],
                    name='Liveness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['speechiness'],
                    name='Speechiness'))

fig.show()


## Analisando a colerração entre as features
---

In [21]:

fig = px.imshow(temp.drop(columns = 'mode').corr(), text_auto = True)
fig.show()

## Criando um Cluster com Generos e comparando com um já classificado:
---

In [22]:
url_dados_generos = 'https://github.com/IgorNascAlves/dados/blob/main/dados_musicas_genero.csv?raw=true'

sessao_spark.sparkContext.addFile(url_dados_generos)
path_dados_file = "file://" + SparkFiles.get("dados_musicas_genero.csv")

dados_generos = sessao_spark.read.csv(path_dados_file, header=True, inferSchema= True)


In [23]:
dados_generos.show()


+----+--------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+-------------------+------------------+---+
|mode|              genres|       acousticness|       danceability|       duration_ms|             energy|    instrumentalness|           liveness|           loudness|         speechiness|             tempo|            valence|        popularity|key|
+----+--------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+-------------------+------------------+---+
|   1|21st century clas...| 0.9793333333333332|0.16288333333333335|160297.66666666663|0.07131666666666665|          0.60683367|             0.3616|-31.514333333333337| 0.04056666666666667|           75.3365|0.10378333333333334| 27.83333333333333| 

In [24]:
dados_generos.count()

2973

In [25]:

dados_generos.select('genres').distinct().count()

2973

In [26]:
len(dados_generos.columns)

14

In [27]:
from pyspark.ml.feature  import VectorAssembler

#iremos transformas as features em um vetor para conseguir usar os modolos de ml do spark

In [28]:
X = dados_generos.columns
X.remove('genres')

dados_generos_vector = VectorAssembler(inputCols=X, outputCol = 'features').transform(dados_generos).select(['features','genres'])

In [29]:
dados_generos_vector.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|features                                                                                                                                                                                                                     |genres                |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|[1.0,0.9793333333333332,0.16288333333333335,160297.66666666663,0.07131666666666665,0.60683367,0.3616,-31.514333333333337,0.04056666666666667,75.3365,0.10378333333333334,27.83333333333333,6.0]                              |21st century classical|
|[1.0,0.4947

In [30]:
from pyspark.ml.feature import StandardScaler

# par aum melhor funcionamento do algoritmo, precisa-se normalizar as feautures  com o standarscaler

In [31]:

scaler = StandardScaler(inputCol = 'features', outputCol='scaled_features')
scaler_model = scaler.fit(dados_generos_vector)
dados_generos_scaler = scaler_model.transform(dados_generos_vector)

In [32]:
dados_generos_scaler.show()

+--------------------+--------------------+--------------------+
|            features|              genres|     scaled_features|
+--------------------+--------------------+--------------------+
|[1.0,0.9793333333...|21st century clas...|[2.68174831000279...|
|[1.0,0.49478,0.29...|               432hz|[2.68174831000279...|
|[1.0,0.762,0.7120...|               8-bit|[2.68174831000279...|
|[1.0,0.6514170195...|                  []|[2.68174831000279...|
|[1.0,0.6765573049...|          a cappella|[2.68174831000279...|
|[1.0,0.45921,0.51...|            abstract|[2.68174831000279...|
|[1.0,0.3421466666...|      abstract beats|[2.68174831000279...|
|[1.0,0.2438540633...|    abstract hip hop|[2.68174831000279...|
|[0.0,0.3229999999...|           accordeon|[0.0,1.0101313736...|
|[1.0,0.446125,0.6...|           accordion|[2.68174831000279...|
|[0.0,0.0679505384...|          acid house|[0.0,0.2125045534...|
|[1.0,0.2569145079...|           acid rock|[2.68174831000279...|
|[1.0,0.00683,0.66...|   

## Usando Tecnica PCA para vizualização:
---
 A analise de componentes principais (PCA) é uma tecninca usada para indentificação de um numero menor de variaveis não correlacionadas conhecidas como componentes principais de um conjunto maior de dados.

A tecninca pe amplamente usada para enfatizar a varicação e captura de padrões fortes em um conjunto de dados. 


É uma forma de redução de dimensionalidade.

para realizar o processo as variaveis devem ser numericas e normaalizadas.


In [33]:
 from pyspark.ml.feature import PCA

In [36]:
pca = PCA( k=2 , inputCol='scaled_features', outputCol='pca_features') 
model_pca =  pca.fit(dados_generos_scaler)

dados_generos_pca = model_pca.transform(dados_generos_scaler)



In [37]:
dados_generos_pca.show()

+--------------------+--------------------+--------------------+--------------------+
|            features|              genres|     scaled_features|        pca_features|
+--------------------+--------------------+--------------------+--------------------+
|[1.0,0.9793333333...|21st century clas...|[2.68174831000279...|[2.50709536688856...|
|[1.0,0.49478,0.29...|               432hz|[2.68174831000279...|[-0.5969679056633...|
|[1.0,0.762,0.7120...|               8-bit|[2.68174831000279...|[-4.1584602762235...|
|[1.0,0.6514170195...|                  []|[2.68174831000279...|[-2.3873448785122...|
|[1.0,0.6765573049...|          a cappella|[2.68174831000279...|[-2.6501218371679...|
|[1.0,0.45921,0.51...|            abstract|[2.68174831000279...|[-1.4965091203367...|
|[1.0,0.3421466666...|      abstract beats|[2.68174831000279...|[-3.9235207721573...|
|[1.0,0.2438540633...|    abstract hip hop|[2.68174831000279...|[-4.6110111098311...|
|[0.0,0.3229999999...|           accordeon|[0.0,1.0101

In [39]:
dados_generos_pca.select('pca_features').show(truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[2.5070953668885667,0.43816913737697943] |
|[-0.5969679056633488,4.981612052751348]  |
|[-4.158460276223561,-0.8366525081079943] |
|[-2.387344878512217,-0.4877989015663404] |
|[-2.6501218371679083,-0.5756819768820474]|
|[-1.496509120336763,1.8644183183717797]  |
|[-3.923520772157324,0.2851835002352836]  |
|[-4.611011109831114,-0.6783790472312378] |
|[-2.837690063084229,-0.5712993716580518] |
|[-2.706690139892783,-1.25937880797083]   |
|[-4.6983313839242875,1.2765569680619446] |
|[-3.375987496679868,0.7560741064307471]  |
|[-5.608998877066021,1.0427311644393213]  |
|[0.2954946352117687,-0.2763864586236301] |
|[-2.5725591062870428,-1.3169815431109795]|
|[-3.4008228020493454,0.5073029625781897] |
|[-4.366720316263419,-0.3364827059771091] |
|[-2.7254698167724003,0.5058604987046365] |
|[-4.958112358381605,1.2627579957290729]  |
|[-3.6934951846422712,1.38227620

## Criando o Pepiline para a preparação dos dados
---

In [40]:
from pyspark.ml import Pipeline

In [41]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'), # vetorização
                                StandardScaler(inputCol='features', outputCol='scaled_features'), #normalização
                                PCA(k=2, inputCol='scaled_features', outputCol='pca_features')])  #rtedução de dimensionalidade


In [42]:
pca_pipeline_model = pca_pipeline.fit(dados_generos)

In [44]:
dados_generos_pca = pca_pipeline_model.transform(dados_generos)

In [45]:
dados_generos.show()

+----+--------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+-------------------+------------------+---+
|mode|              genres|       acousticness|       danceability|       duration_ms|             energy|    instrumentalness|           liveness|           loudness|         speechiness|             tempo|            valence|        popularity|key|
+----+--------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+--------------------+------------------+-------------------+------------------+---+
|   1|21st century clas...| 0.9793333333333332|0.16288333333333335|160297.66666666663|0.07131666666666665|          0.60683367|             0.3616|-31.514333333333337| 0.04056666666666667|           75.3365|0.10378333333333334| 27.83333333333333| 

In [46]:
dados_generos_pca.select('pca_features').show(truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[2.5070953668885667,0.43816913737697943] |
|[-0.5969679056633488,4.981612052751348]  |
|[-4.158460276223561,-0.8366525081079943] |
|[-2.387344878512217,-0.4877989015663404] |
|[-2.6501218371679083,-0.5756819768820474]|
|[-1.496509120336763,1.8644183183717797]  |
|[-3.923520772157324,0.2851835002352836]  |
|[-4.611011109831114,-0.6783790472312378] |
|[-2.837690063084229,-0.5712993716580518] |
|[-2.706690139892783,-1.25937880797083]   |
|[-4.6983313839242875,1.2765569680619446] |
|[-3.375987496679868,0.7560741064307471]  |
|[-5.608998877066021,1.0427311644393213]  |
|[0.2954946352117687,-0.2763864586236301] |
|[-2.5725591062870428,-1.3169815431109795]|
|[-3.4008228020493454,0.5073029625781897] |
|[-4.366720316263419,-0.3364827059771091] |
|[-2.7254698167724003,0.5058604987046365] |
|[-4.958112358381605,1.2627579957290729]  |
|[-3.6934951846422712,1.38227620

## Realizando O K-Means par ao agrupamento
---

In [47]:
from pyspark.ml.clustering import KMeans

In [48]:
SEED = 1224

In [49]:
kmeans = KMeans(featuresCol='pca_features',predictionCol='cluster_pca').setK(5).setSeed(SEED)

In [51]:
model_kmeans = kmeans.fit(dados_generos_pca)

In [53]:
prediction_kmeans = model_kmeans.transform(dados_generos_pca)

In [56]:
prediction_kmeans.select(['pca_features','cluster_pca']).show(truncate=False)

+-----------------------------------------+-----------+
|pca_features                             |cluster_pca|
+-----------------------------------------+-----------+
|[2.5070953668885667,0.43816913737697943] |2          |
|[-0.5969679056633488,4.981612052751348]  |2          |
|[-4.158460276223561,-0.8366525081079943] |4          |
|[-2.387344878512217,-0.4877989015663404] |0          |
|[-2.6501218371679083,-0.5756819768820474]|0          |
|[-1.496509120336763,1.8644183183717797]  |2          |
|[-3.923520772157324,0.2851835002352836]  |4          |
|[-4.611011109831114,-0.6783790472312378] |1          |
|[-2.837690063084229,-0.5712993716580518] |4          |
|[-2.706690139892783,-1.25937880797083]   |0          |
|[-4.6983313839242875,1.2765569680619446] |3          |
|[-3.375987496679868,0.7560741064307471]  |4          |
|[-5.608998877066021,1.0427311644393213]  |1          |
|[0.2954946352117687,-0.2763864586236301] |2          |
|[-2.5725591062870428,-1.3169815431109795]|0    

## Plotando os grupos graficamente
---

In [58]:
from pyspark.ml.functions import vector_to_array  # o vector_to_aray transforma um vector em um array o contrario do vector assembler

In [59]:
pca_features_xy =prediction_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                  .withColumn('y',vector_to_array('pca_features')[1])\
                                  .select(['x','y','cluster_pca','genres'])

In [60]:
pca_features_xy.show()

+-------------------+-------------------+-----------+--------------------+
|                  x|                  y|cluster_pca|              genres|
+-------------------+-------------------+-----------+--------------------+
| 2.5070953668885667|0.43816913737697943|          2|21st century clas...|
|-0.5969679056633488|  4.981612052751348|          2|               432hz|
| -4.158460276223561|-0.8366525081079943|          4|               8-bit|
| -2.387344878512217|-0.4877989015663404|          0|                  []|
|-2.6501218371679083|-0.5756819768820474|          0|          a cappella|
| -1.496509120336763| 1.8644183183717797|          2|            abstract|
| -3.923520772157324| 0.2851835002352836|          4|      abstract beats|
| -4.611011109831114|-0.6783790472312378|          1|    abstract hip hop|
| -2.837690063084229|-0.5712993716580518|          4|           accordeon|
| -2.706690139892783|  -1.25937880797083|          0|           accordion|
|-4.6983313839242875| 1.2

In [61]:
fig = px.scatter(pca_features_xy.toPandas(), x='x',y='y', color ='cluster_pca',hover_data = ['x','y','genres'])
fig.show()

In [72]:
pca_pipeline_model.stages[2].explainedVariance


DenseVector([0.2975, 0.1212])

Os cluster criados satão muito jundos o que não é necessariamente bom. Isto ocorreu pq no processo de pca se perdeu grande parte de informação. uma variavel das criadas explica quase 3 e a outra um pouco mais de uma, de um total de 19 variaveis. 

#Alterando o PCA  
---

In [73]:
# Na definição da função vamos incluir os parâmetros que gostariamos de variar, como quantidade componentes, número de clusters e colunas utilizadas
def cria_pipeline(k_PCA=2, SEED=1224, k_kmeans=5, columns=X, data=dados_generos):


  # Definição do Pipeline
  # Primeiro a vetorização das colunas selecionadas
  # Segundo a padronização dos dados com o StandardScaler
  # Terceiro o PCA com o número k_PCA de componentes
  # Quarto o KMeans com o número k_kmeans de clusters
  pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols = columns, outputCol = 'features'),\
                                StandardScaler(inputCol='features', outputCol='scaled_features'),\
                                PCA(k=k_PCA, inputCol="scaled_features", outputCol='pca_features'),\
                                KMeans(featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED, k=k_kmeans)])

  # ajuste da pipeline com os dados
  model = pca_pipeline.fit(data)

  # transformação dos dados utilizando o modelo da pipeline
  predictions_kmeans = model.transform(data)

  # criando duas novas colunas a partir das primeiras duas componentes do PCA
  pca_features_xs = predictions_kmeans.withColumn("x", vector_to_array("pca_features")[0])\
                   .withColumn("y", vector_to_array("pca_features")[1])\
                   .select(['x', 'y', 'cluster_pca', 'genres'])

  # plotando as duas componentes e o cluster
  fig = px.scatter(
   pca_features_xs.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['x', 'y', 'genres'])
  fig.show()

  # retornando pipeline ajustado
  return model

Valor PCA = 3

In [74]:
cria_pipeline(k_PCA=3)

PipelineModel_34053e27f93c

Valor PCA = 4

In [77]:
cria_pipeline(k_PCA=4)

PipelineModel_a936c2781550

Valor PCA =  5

In [78]:
cria_pipeline(k_PCA=5)

PipelineModel_cead89b68522

# Alterando o numero de cluster com pca = 2
--- 

k=6

In [81]:
cria_pipeline(k_kmeans=6)

PipelineModel_01a9c3e5c9df

k=7

In [82]:
cria_pipeline(k_kmeans=7)

PipelineModel_ab9908bfa831

k=8

In [83]:
cria_pipeline(k_kmeans=8)

PipelineModel_efccb42ce10b

# Fazendo a clusterização pelas musicas
---

In [84]:
X  = dados.columns
X.remove('artists')
X.remove('id')
X.remove('name')
X.remove('artists_song')
X

['valence',
 'year',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'explicit',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'popularity',
 'speechiness',
 'tempo']

In [86]:
dados_encoded_vector = VectorAssembler(inputCols = X, outputCol='features').transform(dados)

In [87]:
dados_encoded_vector.select('features').show()

+--------------------+
|            features|
+--------------------+
|[0.285,2000.0,0.0...|
|[0.613,2000.0,0.1...|
|[0.4,2000.0,0.009...|
|[0.54299999999999...|
|[0.76,2000.0,0.03...|
|[0.941,2000.0,9.9...|
|[0.722,2000.0,0.0...|
|[0.507,2000.0,0.0...|
|[0.861,2000.0,0.0...|
|[0.894,2000.0,0.3...|
|[0.165,2000.0,0.7...|
|[0.53799999999999...|
|[0.654,2000.0,0.0...|
|[0.565,2000.0,0.0...|
|[0.532,2000.0,0.0...|
|[0.866,2000.0,0.3...|
|[0.435,2000.0,0.0...|
|[0.299,2000.0,0.0...|
|[0.915,2000.0,0.0...|
|[0.885,2000.0,0.6...|
+--------------------+
only showing top 20 rows



In [95]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaled = model_scaler.transform(dados_encoded_vector)

In [96]:
dados_musicas_scaled.select('features_scaled').show()

+--------------------+
|     features_scaled|
+--------------------+
|[1.15659667922104...|
|[2.48769741881579...|
|[1.62329358487164...|
|[2.20362104146325...|
|[3.08425781125612...|
|[3.81879815841053...|
|[2.93004492069331...|
|[2.05752461882480...|
|[3.49413944143621...|
|[3.62806116218812...|
|[0.66960860375955...|
|[2.18332987165235...|
|[2.65408501126513...|
|[2.29290218863119...|
|[2.15898046787928...|
|[3.51443061124710...|
|[1.76533177354791...|
|[1.21341195469155...|
|[3.71328407539388...|
|[3.59153705652850...|
+--------------------+
only showing top 20 rows



In [94]:
k = len(X)
k

15

In [97]:
pca = PCA(k=k,inputCol = 'features_scaled', outputCol ='pca_features')
model_pca = pca.fit(dados_musicas_scaled)
dados_musicas_pca = model_pca.transform(dados_musicas_scaled)

In [98]:
model_pca.explainedVariance

DenseVector([0.213, 0.133, 0.0939, 0.0869, 0.0733, 0.0665, 0.0616, 0.0563, 0.0505, 0.0462, 0.0339, 0.0284, 0.0264, 0.0202, 0.0099])

In [99]:
sum(model_pca.explainedVariance)*100

99.99999999999999

In [101]:
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores

[0.21297354127107343,
 0.34597741622955064,
 0.43989805678254446,
 0.5267820578269102,
 0.6001038103117928,
 0.6665719513652388,
 0.7282174213581727,
 0.7845103843321022,
 0.8350002357002613,
 0.8812446547406768,
 0.9151737562703046,
 0.9435370133056037,
 0.9699018569070159,
 0.9900947792885578,
 0.9999999999999999]

In [102]:
import numpy as np

In [106]:
 k = sum(np.array(lista_valores) <= 0.7)
 k

6

In [107]:
pca = PCA(k=k,inputCol = 'features_scaled', outputCol ='pca_features')
model_pca = pca.fit(dados_musicas_scaled)
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaled)

In [108]:
dados_musicas_pca_final.show()

+------------------+----+------------+--------------+------------------+-----------+------------------+--------+--------------------+--------------------+---+--------+-------------------+----+--------------------+----------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+
|           valence|year|acousticness|       artists|      danceability|duration_ms|            energy|explicit|                  id|    instrumentalness|key|liveness|           loudness|mode|                name|popularity|speechiness|             tempo|        artists_song|            features|     features_scaled|        pca_features|
+------------------+----+------------+--------------+------------------+-----------+------------------+--------+--------------------+--------------------+---+--------+-------------------+----+--------------------+----------+-----------+------------------+--------------------+--------------------+--------------------+--

In [111]:
sum(model_pca.explainedVariance)*100

66.65719513652388

## Criando Pipeline para clustering das musicas
---

In [112]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='features_scaled'),
                                PCA(k=6, inputCol='features_scaled', outputCol='pca_features')])


In [113]:
model_pca_pipeline = pca_pipeline.fit(dados)


In [114]:
projection = model_pca_pipeline.transform(dados)


In [115]:
projection.select('pca_features').show(truncate=False, n=5)


+----------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                          |
+----------------------------------------------------------------------------------------------------------------------+
|[-34.71004464775704,-165.36848784906178,-11.163498729833321,-139.11374280854434,-14.152192300931375,6.254422223810391]|
|[-36.49885762774905,-167.85185045616555,-10.729023248907561,-135.904352656713,-13.388401890587229,3.9729807278308606] |
|[-35.460300916940554,-165.99633344577884,-11.361290348241592,-138.24276881247,-13.866654919305779,4.342675920458719]  |
|[-35.56301325520584,-165.59515096480607,-11.460226007031382,-137.52095740570041,-15.56854280392272,4.542073725584271] |
|[-36.54556974907567,-167.37512505802482,-11.881276527236555,-136.27798590243984,-14.05684799034454,3.583390145779156] |
+-------------------------------

In [116]:
kmeans= KMeans( k = 50,featuresCol='pca_features', predictionCol='cluster_pca', seed = SEED)

In [117]:
model_kmeans = kmeans.fit(projection)

In [118]:
projection_kmeans = model_kmeans.transform(projection)

In [119]:
projection_kmeans.select(['pca_features','cluster_pca']).show()

+--------------------+-----------+
|        pca_features|cluster_pca|
+--------------------+-----------+
|[-34.710044647757...|          8|
|[-36.498857627749...|          4|
|[-35.460300916940...|         35|
|[-35.563013255205...|         16|
|[-36.545569749075...|         34|
|[-36.713222290262...|         34|
|[-36.013246178822...|         34|
|[-36.542687712104...|         23|
|[-36.425249009784...|         16|
|[-35.872074915770...|         34|
|[-31.639065936568...|         44|
|[-35.661446890546...|         34|
|[-35.574542234850...|         32|
|[-36.022399748656...|         13|
|[-34.336941950985...|         35|
|[-35.096906055142...|         49|
|[-35.202423054032...|         38|
|[-34.942948671026...|         30|
|[-36.617404517517...|          4|
|[-34.296973387410...|          6|
+--------------------+-----------+
only showing top 20 rows

